# Environment Setup

In [ ]:
!pip install yfinance==0.1.67
!mamba install bs4==4.10.0 -y
!pip install nbformat==4.2.0

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Extracting TSLA Stock Data


In [ ]:
tesla = yf.Ticker('TSLA')
tesla.get_info()

In [ ]:
import yfinance as yf

# Fetch Tesla's stock data
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")

# Convert index to datetime index
tesla_data.index = pd.to_datetime(tesla_data.index)

# Now you can localize the timezone
tesla_data.index = tesla_data.index.tz_localize("UTC")

# Now you can use the DataFrame without encountering the tz_localize error


In [ ]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

## Webscraping Tesla Revenue Data


In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
response = requests.get(url)
html_data = response.text

In [ ]:
soup = BeautifulSoup(html_data, 'html.parser')

In [ ]:
table = soup.find_all("tbody")[1]

rows = table.find_all('tr')

dates = []
revenues = []

for row in rows:
    # Extract columns from the row
    cols = row.find_all('td')
    # Extract Date and Revenue
    date = cols[0].text.strip()
    revenue = cols[1].text.strip().replace('$', '').replace(',', '')  # Remove $ and commas
    # Append data to lists
    dates.append(date)
    revenues.append(revenue)

tesla_revenue = pd.DataFrame({
    'Date': dates,
    'Revenue': revenues
})

print(tesla_revenue)

In [ ]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

In [ ]:
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [ ]:
tesla_revenue.tail()

## Extracting GME Stock Data


In [ ]:
gme = yf.Ticker('GME')

In [ ]:
gme_data = gme.history(period='max')

In [ ]:
gme_data.reset_index(inplace=True)
gme_data.head()

## Webscraping GME Revenue Data


In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
response = requests.get(url)
html_data = response.text

In [ ]:
soup = BeautifulSoup(html_data, 'html.parser')

In [ ]:
soup = BeautifulSoup(html_data, 'html.parser')

table = soup.find_all("tbody")[1]

rows = table.find_all('tr')

dates = []
revenues = []

for row in rows:
    # Extract columns from the row
    cols = row.find_all('td')
    # Extract Date and Revenue
    date = cols[0].text.strip()
    revenue = cols[1].text.strip().replace('$', '').replace(',', '')  # Remove $ and commas
    # Append data to lists
    dates.append(date)
    revenues.append(revenue)

gme_revenue = pd.DataFrame({
    'Date': dates,
    'Revenue': revenues
})

print(gme_revenue)

In [ ]:
gme_revenue.tail()

## Plotting Tesla Stock Graph


In [ ]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

## Plotting GameStop Stock Graph


In [ ]:
make_graph(gme_data, gme_revenue, 'GameStop')